In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path
from boxmot import BotSort


YOLO_RESULTS = r"C:\Users\rouat\Desktop\final\end_to_end_pipeline"
DET_FILE = os.path.join(YOLO_RESULTS, "det.txt")
IMG_FOLDER = r"C:\Users\rouat\Desktop\final\mot\test\img1"

OUTPUT_FOLDER = os.path.join(YOLO_RESULTS, "tracking_results")
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

REID_WEIGHTS = r"osnet_x0_25_msmt17.pt"
OUTPUT_VIDEO = os.path.join(OUTPUT_FOLDER, "tracking_output.avi")
OUTPUT_TXT = os.path.join(OUTPUT_FOLDER, "tracking_results_yolo.txt")

def load_detections(det_file):
    det_data = {}
    with open(det_file, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split(',')
            frame = int(parts[0])
            x = float(parts[2])
            y = float(parts[3])
            w = float(parts[4])
            h = float(parts[5])
            conf = float(parts[6])
            cls = int(parts[7])
            det = [x, y, w, h, conf, cls]
            det_data.setdefault(frame, []).append(det)
    return det_data

def convert_to_bot_format(dets):
    arr = []
    for d in dets:
        x1 = d[0]
        y1 = d[1]
        x2 = x1 + d[2]
        y2 = y1 + d[3]
        conf = d[4]
        cls = d[5]
        arr.append([x1, y1, x2, y2, conf, cls])
    return np.array(arr, dtype=np.float32)

def get_color_for_id(track_id):
    np.random.seed(int(track_id))
    color = np.random.randint(0, 255, 3)
    return tuple(int(c) for c in color)

def main():
    det_data = load_detections(DET_FILE)
    tracker = BotSort(
        reid_weights=Path(REID_WEIGHTS),
        device='cuda:0',
        half=False
    )

    frame_nums = sorted(det_data.keys())

    # Initialisation du VideoWriter 
    first_frame_path = os.path.join(IMG_FOLDER, f"{frame_nums[0]:06d}.jpg")
    first_img = cv2.imread(first_frame_path)
    if first_img is None:
        print(f"Première image introuvable : {first_frame_path}")
        return

    height, width = first_img.shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, 30, (width, height))
    if not out.isOpened():
        print("Erreur : impossible de créer la vidéo.")
        return

    with open(OUTPUT_TXT, "w") as f:
        for frame_num in frame_nums:
            img_path = os.path.join(IMG_FOLDER, f"{frame_num:06d}.jpg")
            img = cv2.imread(img_path)
            if img is None:
                print(f"Image frame {frame_num} introuvable : {img_path}")
                continue

            dets = det_data.get(frame_num, [])
            dets_np = convert_to_bot_format(dets)

            tracks = tracker.update(dets_np, img)

            for t in tracks:
                x1, y1, x2, y2, track_id, cls, conf = t[:7]
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                w, h = x2 - x1, y2 - y1

                f.write(f"{frame_num},{int(track_id)},{x1},{y1},{w},{h},{conf},{int(cls)}\n")

                color = get_color_for_id(track_id)
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, f"ID:{int(track_id)} Cls:{int(cls)}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            out.write(img)

            display_img = cv2.resize(img, (960, 540))
            cv2.imshow("Tracking", display_img)
            key = cv2.waitKey(1)
            if key == 27:
                break

    out.release()
    cv2.destroyAllWindows()
    print(f" Vidéo sauvegardée dans : {OUTPUT_VIDEO}")
    print(f" Résultats texte sauvegardés dans : {OUTPUT_TXT}")

if __name__ == "__main__":
    main()


2025-09-27 11:17:50.078 | INFO     | boxmot.utils.torch_utils:select_device:78 - Yolo Tracking v15.0.2 🚀 Python-3.9.23 torch-2.5.1+cu121
CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
2025-09-27 11:17:50.081 | INFO     | boxmot.appearance.backends.base_backend:download_model:145 - [PID 12100] Found existing ReID weights at osnet_x0_25_msmt17.pt; skipping download.
2025-09-27 11:17:50.170 | SUCCESS  | boxmot.appearance.reid.registry:load_pretrained_weights:64 - Loaded pretrained weights from osnet_x0_25_msmt17.pt


 Vidéo sauvegardée dans : C:\Users\rouat\Desktop\final\end_to_end_pipeline\tracking_results\tracking_output.avi
 Résultats texte sauvegardés dans : C:\Users\rouat\Desktop\final\end_to_end_pipeline\tracking_results\tracking_results_yolo.txt
